In [1]:
import re
import os
from striprtf.striprtf import rtf_to_text

In [2]:
from transformers import pipeline

2023-05-27 20:47:25.883456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [35]:
nlp = pipeline('question-answering', model='AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru', tokenizer='AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru')

In [8]:
doc_names = [
    'Москва/Решение от 10102022 по делу  А40 148161 2022.rtf',
    'Москва/Решение от 11112022 по делу  А40 161479 2022.rtf',
    'Москва/Решение от 12102022 по делу  А40 155607 2022.rtf',
    'Москва/Решение от 13102022 по делу  А40 168386 2021.rtf',
    'Москва/Решение от 14102022 по делу  А40 158698 2022.rtf',
    'Красноярский край/Решение от 01112022 по делу  А33 2122 2022.rtf',
    'Красноярский край/Решение от 03112022 по делу  А33 21788 2022.rtf',
    'Красноярский край/Решение от 06102022 по делу  А33 3874 2022_лимит ответственности - отказ в снижении неустойки.rtf',
    'Красноярский край/Решение от 06122021 по делу  А33 24535 2021.rtf',
    'Красноярский край/Решение от 18062021 по делу  А33 796 2021.rtf',
    'Новосибирская область/Решение от 08092022 по делу  А45 10799 2022_лимит ответственности - отказ в снижении.rtf',
    'Новосибирская область/Решение от 10102022 по делу  А45 14812 2022.rtf',
    'Новосибирская область/Решение от 14102022 по делу  А45 17688 2022.rtf',
    'Новосибирская область/Решение от 15092022 по делу  А45 16196 2022.rtf',
    'Новосибирская область/Решение от 20102022 по делу  А45 16697 2022_лимит ответственности - отказ в снижении.rtf',
]

In [11]:
def get_documents(root):
    docs = {}
    for doc_name in os.listdir(root):
        if doc_name.endswith('.rtf'):
            with open(root + doc_name) as f:
                text = rtf_to_text(f.read())
                docs[doc_name] = text
    return docs

In [18]:
def get_document_sample(paths):
    docs = {}
    for doc_name in paths:
        if doc_name.endswith('.rtf'):
            with open(doc_name) as f:
                text = rtf_to_text(f.read(), errors='ignore')
                docs[doc_name] = text
    return docs

In [19]:
docs = get_document_sample(doc_names)

In [20]:
docs.keys()

dict_keys(['Москва/Решение от 10102022 по делу  А40 148161 2022.rtf', 'Москва/Решение от 11112022 по делу  А40 161479 2022.rtf', 'Москва/Решение от 12102022 по делу  А40 155607 2022.rtf', 'Москва/Решение от 13102022 по делу  А40 168386 2021.rtf', 'Москва/Решение от 14102022 по делу  А40 158698 2022.rtf', 'Красноярский край/Решение от 01112022 по делу  А33 2122 2022.rtf', 'Красноярский край/Решение от 03112022 по делу  А33 21788 2022.rtf', 'Красноярский край/Решение от 06102022 по делу  А33 3874 2022_лимит ответственности - отказ в снижении неустойки.rtf', 'Красноярский край/Решение от 06122021 по делу  А33 24535 2021.rtf', 'Красноярский край/Решение от 18062021 по делу  А33 796 2021.rtf', 'Новосибирская область/Решение от 08092022 по делу  А45 10799 2022_лимит ответственности - отказ в снижении.rtf', 'Новосибирская область/Решение от 10102022 по делу  А45 14812 2022.rtf', 'Новосибирская область/Решение от 14102022 по делу  А45 17688 2022.rtf', 'Новосибирская область/Решение от 15092022

In [22]:
# print(docs['Решение от 24102022 по делу  А56 76859 2022.rtf'])

In [23]:
conditions_part = re.compile(
    '\\n' + '\\s?'.join('установил:') + '(.+?)\n' + '\\s?'.join('решил:') + '\n',
    flags=(re.DOTALL + re.IGNORECASE)
)

In [16]:
text = docs['Решение от 24102022 по делу  А56 76859 2022.rtf']

In [25]:
money = '[0-9 .,]+р.+?'
contract_price_par = re.compile(f'\n(.+?[Оо]бщ[аяей]+\\sстоимостью?\\s.+?{money}'
                                f'|.*?[Сс]умма\\sдоговора\\s.+?{money}'
                                f'|.*?[Оо]тветчик\\sобязался\\sпоставить\\s.+?на\\sсумму\\s{money}'
                                f'|.*?[Сс]тоимость\\sработы?\\sсостав[ляетиа]+\\s{money}'
                                f'|.+?100%\\sот\\sстоимости\\s.+?а\\sименно.+?{money}'
                                f'|.*?[Цц]ена\\sдоговора.+?составляет.+?{money}'
                                f'|.+?на\\sобщую\\sсумму\\s.+?{money}'
                                f'|.+?принял.*?\\sобязательство\\sуплатить.*?\\sсумму.+?{money})\n')
penalty_par = re.compile('\n.*?[Нн]еустойк.+?\n')
debt_par = re.compile(f'\n(.*?[Сс]тоимость\\sне(до)?поставленного\\sтовара.+?{money})\n')

In [26]:
def get_contract_price(text):
    par = contract_price_par.findall(text)
    if par:
        res = nlp({
            'question': 'Какая цена договора?',
            'context': ' '.join(par)
        })
        return res['answer']

In [28]:
# get_contract_price(text)

In [29]:
def get_penalty(text):
    par = penalty_par.findall(text)
    if par:
        res = nlp({
            'question': 'Какой размер неустойки?',
            'context': ' '.join(par)
        })
        return res['answer']

In [180]:
def get_liability(text):
    res = nlp({
        'question': 'Какая сумма задолженности?',
        'context': text
    })
    return res['start'], res['end'], res['answer']

In [219]:
def get_deadline(text):
    res = nlp({
        'question': 'Когда истёк срок исполнения договора?',
        'context': text
    })
    return res['start'], res['end'], res['answer'] 

In [47]:
def get_conditions(text):
    conditions_match = conditions_part.search(text)
    if conditions_match:
        conditions = conditions_match.group(1).strip()
        offset = text.index(conditions)
        return offset, conditions
    
    return 0, text

In [48]:
def extract_all_entities(text):
    entities = []
    offset, conditions = get_conditions(text)
    contract_price = get_contract_price(conditions)
    if contract_price:
        entities.append([
            text.index(contract_price),
            text.index(contract_price) + len(contract_price),
            'Цена контракта',
#             contract_price
        ])
    penalty = get_penalty(conditions)
    if penalty:
        entities.append([
            text.index(penalty),
            text.index(penalty) + len(penalty),
            'Заявленный размер неустойки (цифра)',
#             penalty
        ])
#     debt = get_liability(conditions)
#     entities.append([debt[0] + offest, debt[1] + offest, 'Сумма долга',
# #                      debt[2]
#                     ])
#     deadline = get_deadline(conditions)
#     entities.append([deadline[0] + offest, deadline[1] + offest, 'Срок исполнения договора',
# #                      deadline[2]
#                     ])
    return entities

In [124]:
# docs.keys()

In [125]:
# print(docs['Решение от 24102022 по делу  А56 76859 2022.rtf'])

In [49]:
text = docs['Москва/Решение от 10102022 по делу  А40 148161 2022.rtf']

In [50]:
extract_all_entities(docs['Москва/Решение от 10102022 по делу  А40 148161 2022.rtf'])

[[6286, 6308, 'Заявленный размер неустойки (цифра)']]

In [51]:
text[6286: 6308]

' 125 775 руб. 20 коп.,'

In [93]:
# contract_price_par.findall(text)

In [160]:
contract_sum_questions = [
    'На какую сумму подписали договор?', 'Какая общая сумма контракта?', 'Какая цена договора?',
]

In [42]:
from tqdm import tqdm

In [52]:
data = []
for name, text in tqdm(docs.items()):
    data.append({
        'text': text,
        'label': extract_all_entities(text)
    })

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [02:09<00:00,  8.65s/it]


In [53]:
import json

In [161]:
# with open('spb_docs_processed.json', 'w') as f:
#     json.dump(data[:5], f)

In [54]:
with open('diversified_dataset.jsonl', 'w') as f:
    for doc in data:
        f.write(json.dumps(doc) + '\n')

In [32]:
# QA_input = {
#     'question': 'Какая общая сумма контракта?',
#     'context': text
# }
# res = nlp(QA_input)
# res

In [165]:
# QA_input = {
#     'question': 'Какая цена договора?',
#     'context': text
# }
# res = nlp(QA_input)
# res

In [157]:
# a = ' '.join(['Во исполнение договора стороны подписали спецификацию № 1 от 28.12.2017 (далее - Спецификация) на поставку товара общей стоимостью 73 632 148,16 руб. на условиях доставки груза покупателю. ', 'Обязательство по уплате аванса Покупатель исполнил платежными поручениями от 20.02.2018 №1854, от 02.02.2018 №1727, на общую сумму 22 089 644,45 руб. Платежным поручением от 30.05.2018 №3196 Покупатель уплатил дополнительный аванс в размере 5 000 000 руб. Общая сумма уплаченного аванса составила 27 089 644,45 руб. Срок поставки всего объема товара наступил 21.06.2018.'])
# QA_input = {
#     'question': 'Какая цена договора?',
#     'context': a
# }
# res = nlp(QA_input)
# res